In [109]:
import numpy as np
import sys, operator

In [130]:
class Fitness:
    def __init__(self, path, dist_mtx):
        self.chromosome = path
        self.dist_mtx = dist_mtx
        self.distance = 0
        self.fitness = 0.0
        
    def path_distance(self):
        if self.distance == 0:
            self.distance = sum([self.dist_mtx[self.chromosome[i]][self.chromosome[i+1]] 
                                 for i in range(len(self.chromosome)-1)])
        return self.distance
    
    def rank_path(self, population):
        rank = {}
        for chromosome in population:
            path = "".join(str(elem)+"-" for elem in chromosome)
            rank[path] = sum([self.dist_mtx[chromosome[i]][chromosome[i+1]] 
                                 for i in range(len(chromosome)-1)])
        return dict(sorted(rank.items(), key = operator.itemgetter(1)))

In [149]:
def init_population(n_population, dist_mtx):
    path = np.arange(len(dist_mtx[0]))
    pop = np.array([np.random.permutation(path) for _ in range(n_population)])
    return np.array([np.append(p, p[0]) for p in pop])

def select(rank, elite_size):
    keys = list(rank.keys())
    selections = []
    for i in range(elite_size):
        selections.append(keys[i])
    return selections

def order_crossover(p1, p2):
    n = len(p1)
    cut_1 = round(n/3)
    cut_2 = round(2*n/3) + 1
    o1_mid = p1[cut_1: cut_2]
    o2_mid = p2[cut_1: cut_2]
    seq_1 = p1[cut_2:] + p1[:cut_2]
    seq_2 = p2[cut_2:] + p2[:cut_2]
    for i in o1_mid:
        seq_2 = seq_2.replace(i, '')
    for i in o2_mid:
        seq_1 = seq_1.replace(i, '')
    o1 = seq_2[-cut_1:] + o1_mid + seq_2[:n-cut_2]
    o2 = seq_1[-cut_1:] + o2_mid + seq_1[:n-cut_2]
    return o1, o2

def swap(chromosome):
    a, b = np.random.choice(len(chromosome), 2)
    x = chromosome[a]
    y = chromosome[b] 
    chromosome = chromosome.replace(x, '$x') 
    chromosome = chromosome.replace(y, '$y')
    chromosome = chromosome.replace('$x', y)
    chromosome = chromosome.replace('$y', x)
    return chromosome

def mutation(chromosome, p_mutation):
    if np.random.rand() < p_mutation:
        return swap(chromosome)
    return chromosome

def create_new_population(rank, p_crossover, p_mutation):
    new_pop = []
    n_population = len(rank)
    elite_size = round(n_population*p_crossover)
    parents = select(rank, elite_size)
    new_pop.extend(parents)
    m = n_population - elite_size 
    for i in range(elite_size-1):
        o1, o2 = order_crossover(parents[i], parents[i+1])
        o1 = mutation(o1, p_mutation)
        o2 = mutation(o2, p_mutation)
        new_pop.extend([o1, o2])
    new_pop = new_pop[:n_population]
    pop = []
    for chromosome in new_pop:
        chromosome = chromosome.replace("-", "")
        pop.append([int(i) for i in chromosome])
    return pop

def genetic_tsp(n_population, dist_mtx, p_crossover, p_mutation, generation):
    shortest = sys.maxsize
    bestpath = ''
    i = 0
    gen = 0
    while (i < 100) and (gen < generation):
        pop = init_population(n_population, dist_mtx)
        fitness = Fitness(pop[0], dist_mtx)
        rank = fitness.rank_path(pop)
        if list(rank.values())[0] < shortest:
            shortest = list(rank.values())[0]
            bestpath = list(rank.keys())[0]
        else:
            i += 1
        pop = create_new_population(rank, p_crossover, p_mutation)
        gen += 1
    return shortest, bestpath

In [150]:
dm = np.array([
        [0, 2, 9999, 12, 5],
        [2, 0, 4, 8, 9999],
        [9999, 4, 0, 3, 3],
        [12, 8, 3, 0, 10],
        [5, 9999, 3, 10, 0]
    ])
genetic_tsp(10, dm, 0.2, 0.2, 200)

(21, '0-4-2-3-1-0-')

In [151]:
locations = {0:'New York', 1:'Los Angeles', 2:'Chicago', 3:'Minneapolis', 4:'Denver', 5:'Dallas', 6:'Seattle',
             7:'Boston', 8:'San Francisco', 9:'St. Louis', 10:'Houston', 11:'Phoenix', 12:'Salt Lake City'}
distance_matrix = np.array( [
    [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
    [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
    [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
    [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
    [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
    [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
    [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
    [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
    [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
    [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
    [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
    [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
    [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
] )
    
genetic_tsp(500, distance_matrix, 0.2, 0.2, 2000)

(8031, '5-4-11-12-1-8-6-3-2-0-7-9-10-5-')

In [152]:
distance_matrix = np.array([
        [0.00, 28.02, 17.12, 27.46, 46.07],
        [28.02, 0.00, 34.00, 25.55, 25.55],
        [17.12, 34.00, 0.00, 18.03, 57.38],
        [27.46, 25.55, 18.03, 0.00, 51.11],
        [46.07, 25.55, 57.38, 51.11, 0.00],
    ])

genetic_tsp(50, distance_matrix, 0.3, 0.6, 2000)

(132.32, '4-1-3-2-0-4-')